In [ ]:
from surface_roughness import Surface
import numpy as np
from scipy.stats import shapiro, kstest, norm, gaussian_kde, probplot
import matplotlib.pyplot as plt

surface = Surface(path='example_surface.stl')

In [ ]:
v0 = surface.points[surface.triangles[:,0]]
v1 = surface.points[surface.triangles[:,1]]
v2 = surface.points[surface.triangles[:,2]]

v1v0 = v1 - v0
v2v0 = v2 - v0

normals = np.cross(v1v0,v2v0,axisa=1,axisb=1)

areas = np.linalg.norm(normals,axis=1)
print(areas)
normals /= areas[:,np.newaxis]
print(normals)

In [ ]:
shear_dir = np.array([1,0,0])

proj = np.zeros_like(normals)
proj[:,0] = normals[:,0] - (shear_dir[1]**2*normals[:,0] - shear_dir[0]*shear_dir[1]*normals[:,1])
proj[:,1] = normals[:,1] - (shear_dir[0]*shear_dir[1]*normals[:,0] + shear_dir[0]**2 * normals[:,1])
proj[:,2] = normals[:,2]

proj /= np.linalg.norm(proj,axis=1)[:,np.newaxis]
app_dip = np.arccos(proj[:,0]*shear_dir[0]+proj[:,1]*shear_dir[1])-np.pi/2
print(app_dip)

In [ ]:
stat, pval = shapiro(app_dip)
print(len(app_dip))
print(f"Shapiro-wilk stat: {stat} p-value: {pval}")

ks_stat, ks_pval = kstest(app_dip,'norm')
print(f"Kolmogorov-Smirnov Stat: {ks_stat} P-value: {ks_pval}")

mean = np.mean(app_dip)
stdev = np.std(app_dip)

print(f"Mean: {mean} Std: {stdev}")
print(f"Absmean: {np.mean(np.abs(app_dip))} Std: {np.std(np.abs(app_dip))}")

x = np.linspace(app_dip.min(),app_dip.max(),500)
kern = gaussian_kde(app_dip)

plt.hist(app_dip,100,density=True)
plt.twinx()
plt.plot(x,norm.pdf(x,mean,stdev))
plt.plot(x,kern.evaluate(x))
ax = plt.gca()
ax.set_ylim(ymin=0)

In [ ]:
probplot(app_dip,dist="norm",plot=plt)